<a href="https://colab.research.google.com/github/TheoBacqueyrisse/Graph-Neural-Networks/blob/main/GNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Basic Graph Neural Network Architecture**

In [1]:
# Let us first clone the GitHub repository
%%capture
!git clone https://github.com/TheoBacqueyrisse/Graph-Neural-Networks.git

In [2]:
# Install dependencies
%%capture
%cd Graph-Neural-Networks
!pip install -r requirements.txt

In [3]:
from utils import *

## GNN Module Architecture

In [5]:
EMBEDDING_SIZE = 32

class GNN(torch.nn.Module):
    def __init__(self):
      super(GNN, self).__init__()

      # Care about the design of the NN here
      self.initial_conv = GATConv(in_channels = 1, out_channels = EMBEDDING_SIZE)
      self.conv_layer1 = GATConv(in_channels = EMBEDDING_SIZE, out_channels = EMBEDDING_SIZE)
      self.conv_layer2 = GATConv(in_channels = EMBEDDING_SIZE, out_channels = EMBEDDING_SIZE)

      self.pooling = gmp
      self.out = Linear(in_features = EMBEDDING_SIZE, out_features = 1)

    def forward(self, x, edge_index, batch_index):

      y = self.initial_conv(x, edge_index)
      y = F.sigmoid(y)
      y = F.dropout(y, p = 0.2)

      y = self.conv_layer1(y, edge_index)
      y = F.sigmoid(y)
      y = F.dropout(y, p = 0.1)

      y = self.conv_layer2(y, edge_index)
      y = F.sigmoid(y)

      y = self.pooling(y, batch_index)

      out = self.out(y)

      return out, y

model = GNN()
print(model)

GNN(
  (initial_conv): GATConv(1, 32, heads=1)
  (conv_layer1): GATConv(32, 32, heads=1)
  (conv_layer2): GATConv(32, 32, heads=1)
  (out): Linear(in_features=32, out_features=1, bias=True)
)


## Configuration

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

NUM_EPOCHS = 40

loss_function = L1Loss()

optimizer = Adam(params = model.parameters(), lr = 0.001)

scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=20, min_lr=0.00001)

In [10]:
NB_GRAPHS_PER_BATCH = 64

train = ZINC('/content/Graph-Neural-Networks/data', split = 'train')
val = ZINC('/content/Graph-Neural-Networks/data', split = 'val')
test = ZINC('/content/Graph-Neural-Networks/data', split = 'test')

train_loader = DataLoader(train,
                          batch_size = NB_GRAPHS_PER_BATCH,
                          shuffle = True)

val_loader = DataLoader(val,
                        batch_size = NB_GRAPHS_PER_BATCH,
                        shuffle = False)

test_loader = DataLoader(test,
                         batch_size = NB_GRAPHS_PER_BATCH,
                         shuffle = False)

print("Number of Batches in Train Loader :", len(train_loader))
print("Number of Batches in Val Loader :", len(val_loader))
print("Number of Batches in Test Loader :", len(test_loader))

Number of Batches in Train Loader : 3438
Number of Batches in Val Loader : 382
Number of Batches in Test Loader : 79


## Train and Test Functions 🚀

In [11]:
def train(train_loader, val_loader):
  for epoch in range(NUM_EPOCHS+1):

    model.train()
    tot_train_loss = 0.0

    for batch in train_loader:

      # Use GPU
      batch.to(device)

      # Set Gradient values to 0
      optimizer.zero_grad()

      pred, y = model(batch.x.float(), batch.edge_index, batch.batch)

      # Compute Loss and Gradients
      loss = loss_function(pred, batch.y.view(-1, 1).float())
      loss.backward()
      tot_train_loss += loss.item()

      optimizer.step()

    average_train_loss = tot_train_loss / len(train_loader)

    model.eval()
    with torch.no_grad():
        tot_val_loss = 0.0

        for val_batch in val_loader:
            val_batch.to(device)

            val_pred, val_y = model(val_batch.x.float(), val_batch.edge_index, val_batch.batch)
            val_loss = loss_function(val_pred, val_batch.y.view(-1, 1).float())

            tot_val_loss += val_loss.item()

        average_val_loss = tot_val_loss / len(val_loader)

    scheduler.step(average_val_loss)

    # if epoch % 10 == 0:
    print(f"Epoch {epoch} -> Train Loss: {average_train_loss:.4f} - Val Loss: {average_val_loss:.4f}")


def test(test_loader):
  model.eval()
  with torch.no_grad():
      tot_test_loss = 0.0

      for test_batch in test_loader:
          test_batch.to(device)

          test_pred, test_y = model(test_batch.x.float(), test_batch.edge_index, test_batch.batch)
          test_loss = loss_function(test_pred, test_batch.y.view(-1, 1).float())

          tot_test_loss += test_loss.item()

      average_test_loss = tot_test_loss / len(test_loader)

  print(f"Test Loss: {average_test_loss:.4f}")

## Model Training and Evaluation ⚡



In [12]:
train(train_loader, val_loader)

Epoch 0 -> Train Loss: 1.2687 - Val Loss: 1.1884
Epoch 1 -> Train Loss: 1.1641 - Val Loss: 1.1280
Epoch 2 -> Train Loss: 1.1097 - Val Loss: 1.0978
Epoch 3 -> Train Loss: 1.0842 - Val Loss: 1.0671
Epoch 4 -> Train Loss: 1.0610 - Val Loss: 1.0539
Epoch 5 -> Train Loss: 1.0513 - Val Loss: 1.0455
Epoch 6 -> Train Loss: 1.0418 - Val Loss: 1.0354
Epoch 7 -> Train Loss: 1.0330 - Val Loss: 1.0308
Epoch 8 -> Train Loss: 1.0248 - Val Loss: 1.0190
Epoch 9 -> Train Loss: 1.0208 - Val Loss: 1.0136
Epoch 10 -> Train Loss: 1.0156 - Val Loss: 1.0082
Epoch 11 -> Train Loss: 1.0106 - Val Loss: 1.0074
Epoch 12 -> Train Loss: 1.0056 - Val Loss: 0.9996
Epoch 13 -> Train Loss: 0.9891 - Val Loss: 0.9762
Epoch 14 -> Train Loss: 0.9693 - Val Loss: 0.9575
Epoch 15 -> Train Loss: 0.9560 - Val Loss: 0.9411
Epoch 16 -> Train Loss: 0.9479 - Val Loss: 0.9354
Epoch 17 -> Train Loss: 0.9392 - Val Loss: 0.9310
Epoch 18 -> Train Loss: 0.9311 - Val Loss: 0.9278
Epoch 19 -> Train Loss: 0.9249 - Val Loss: 0.9182
Epoch 20 -

In [ ]:
test(test_loader)

Test Loss: 0.8807
